In [1]:
# So get a list of players at the start of a quarter for each time (home 1-5 and away 1-5)
# I think I'll need to loop through each row of the dataframe and modify the list of players for each sub event (home and away)
#     Will also calculate the margin, score, and seconds for each

In [2]:
import requests
import json
import pandas as pd
import numpy as np
from sklearn.preprocessing import MultiLabelBinarizer
pd.set_option('display.max_columns',500)

In [3]:
play_by_play_url = 'https://stats.nba.com/stats/playbyplayv2/?gameId=0041700404&&startPeriod=0&endPeriod=14' #
games_url = 'https://stats.nba.com/stats/leaguegamelog?\
Counter=1000&DateFrom=&DateTo=&Direction=DESC&LeagueID=00\
&PlayerOrTeam=T&Season=2021-22&SeasonType=Playoffs&Sorter=DATE' #
players_game_url = 'https://stats.nba.com/stats/boxscoreadvancedv2/?gameId=0041700404&startPeriod=0&endPeriod=14&startRange=7205&endRange=14395&rangeType=2'

In [4]:
# Headers for querying NBA JSON data api
header_data  = {
    'Connection': 'keep-alive',
    'Accept': 'application/json, text/plain, */*',
    'x-nba-stats-token': 'true',
    'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_14_6) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/79.0.3945.130 Safari/537.36',
    'x-nba-stats-origin': 'stats',
    'Sec-Fetch-Site': 'same-origin',
    'Sec-Fetch-Mode': 'cors',
    'Referer': 'https://stats.nba.com/',
    'Accept-Encoding': 'gzip, deflate, br',
    'Accept-Language': 'en-US,en;q=0.9',
}

In [5]:
# code for getting starting quarter lineups from here: https://github.com/rd11490/NBA-Play-By-Play-Example
# endpoints
def play_by_play_url(game_id):
    return "https://stats.nba.com/stats/playbyplayv2/?gameId={0}&startPeriod=0&endPeriod=14".format(game_id)

def advanced_boxscore_url(game_id, start, end):
    return "https://stats.nba.com/stats/boxscoreadvancedv2/?gameId={0}&startPeriod=0&endPeriod=14&startRange={1}&endRange={2}&rangeType=2".format(game_id, start, end)

def extract_data(url):
    resp = requests.get(url, headers=header_data).json()
#     resp = json.loads(r.data)
    results = resp['resultSets'][0]
    headers = results['headers']
    rows = results['rowSet']
    frame = pd.DataFrame(rows)
    frame.columns = headers
    return frame

def calculate_time_at_period(period):
    if period > 5:
        return (720 * 4 + (period - 5) * (5 * 60)) * 10
    else:
        return (720 * (period - 1)) * 10
    
def split_subs(df, tag):
    subs = df[[tag, 'PERIOD', 'EVENTNUM']]
    subs['SUB'] = tag
    subs.columns = ['PLAYER_ID', 'PERIOD', 'EVENTNUM', 'SUB']
    return subs

game_id = "0042100236"
frame = extract_data(play_by_play_url(game_id)).fillna('')

substitutionsOnly = frame[frame["EVENTMSGTYPE"] == 8][['PERIOD', 'EVENTNUM', 'PLAYER1_ID', 'PLAYER2_ID']]
substitutionsOnly.columns = ['PERIOD', 'EVENTNUM', 'OUT', 'IN']

subs_in = split_subs(substitutionsOnly, 'IN')
subs_out = split_subs(substitutionsOnly, 'OUT')

full_subs = pd.concat([subs_out, subs_in], axis=0).reset_index()[['PLAYER_ID', 'PERIOD', 'EVENTNUM', 'SUB']]
first_event_of_period = full_subs.loc[full_subs.groupby(by=['PERIOD', 'PLAYER_ID'])['EVENTNUM'].idxmin()]
players_subbed_in_at_each_period = first_event_of_period[first_event_of_period['SUB'] == 'IN'][['PLAYER_ID', 'PERIOD', 'SUB']]

periods = players_subbed_in_at_each_period['PERIOD'].drop_duplicates().values.tolist()

frames = []
for period in periods:

    low = calculate_time_at_period(period) + 5
    high = calculate_time_at_period(period + 1) - 5
    boxscore = advanced_boxscore_url(game_id, low, high)
    boxscore_players = extract_data(boxscore)[['PLAYER_NAME', 'PLAYER_ID', 'TEAM_ABBREVIATION']]
    boxscore_players['PERIOD'] = period

    players_subbed_in_at_period = players_subbed_in_at_each_period[players_subbed_in_at_each_period['PERIOD'] == period]

    joined_players = pd.merge(boxscore_players, players_subbed_in_at_period, on=['PLAYER_ID', 'PERIOD'], how='left')
    joined_players = joined_players[pd.isnull(joined_players['SUB'])][['PLAYER_NAME', 'PLAYER_ID', 'TEAM_ABBREVIATION', 'PERIOD']]
    frames.append(joined_players)

out = pd.concat(frames)
out

,PLAYER_NAME,PLAYER_ID,TEAM_ABBREVIATION,PERIOD
0,Dillon Brooks,1628415,MEM,1
1,Jaren Jackson Jr.,1628991,MEM,1
2,Steven Adams,203500,MEM,1
3,Desmond Bane,1630217,MEM,1
4,Tyus Jones,1626145,MEM,1
9,Andrew Wiggins,203952,GSW,1
10,Draymond Green,203110,GSW,1
11,Kevon Looney,1626172,GSW,1
12,Klay Thompson,202691,GSW,1
13,Stephen Curry,201939,GSW,1


In [6]:
games = extract_data(games_url)
games = games[games['MATCHUP'].str.contains('vs.')][['GAME_ID','MATCHUP']]
games['HOME']=games['MATCHUP'].str.split(' vs. ').str.get(0)
games['AWAY']=games['MATCHUP'].str.split(' vs. ').str.get(1)
games

,GAME_ID,MATCHUP,HOME,AWAY
0,0042100227,PHX vs. DAL,PHX,DAL
3,0042100217,BOS vs. MIL,BOS,MIL
4,0042100236,GSW vs. MEM,GSW,MEM
6,0042100216,MIL vs. BOS,MIL,BOS
8,0042100226,DAL vs. PHX,DAL,PHX
...,...,...,...,...
128,0042100101,MIA vs. ATL,MIA,ATL
130,0042100131,PHI vs. TOR,PHI,TOR
132,0042100171,DAL vs. UTA,DAL,UTA
135,0042100161,GSW vs. DEN,GSW,DEN


In [7]:
out = out.groupby(['TEAM_ABBREVIATION','PERIOD'])['PLAYER_ID'].apply(lambda x: ','.join(map(str, x))).reset_index()
out

,TEAM_ABBREVIATION,PERIOD,PLAYER_ID
0,GSW,1,"203952,203110,1626172,202691,201939"
1,GSW,2,"203110,202691,1629673,1627814,202357"
2,GSW,3,"203952,203110,1626172,202691,201939"
3,GSW,4,"203952,1626172,202691,1629673,1627814"
4,MEM,1,"1628415,1628991,203500,1630217,1626145"
5,MEM,2,"1628991,1630217,1630533,1629634,1629001"
6,MEM,3,"1628415,1628991,203500,1630217,1626145"
7,MEM,4,"203500,1630217,203937,1630533,1629001"


In [8]:
frame['HOME']=games[games['GAME_ID']==game_id]['HOME'].values[0]
frame['AWAY']=games[games['GAME_ID']==game_id]['AWAY'].values[0]
frame = frame.merge(out,how='left',left_on=['HOME','PERIOD'],right_on=['TEAM_ABBREVIATION','PERIOD']).drop('TEAM_ABBREVIATION',axis=1)
frame.rename({'PLAYER_ID':'HOME_PLAYER_IDS'},axis=1,inplace=2)
frame = frame.merge(out,how='left',left_on=['AWAY','PERIOD'],right_on=['TEAM_ABBREVIATION','PERIOD']).drop('TEAM_ABBREVIATION',axis=1)
frame.rename({'PLAYER_ID':'AWAY_PLAYER_IDS'},axis=1,inplace=2)
frame['HOME_SUB']=frame['HOMEDESCRIPTION'].str.contains('SUB')
frame['AWAY_SUB']=frame['VISITORDESCRIPTION'].str.contains('SUB')
frame['QUARTER_START']=frame['NEUTRALDESCRIPTION'].str.contains('Start')
frame['PLAYER_CHANGE']=np.where((frame['QUARTER_START']==True)|(frame['AWAY_SUB']==True)|(frame['HOME_SUB']==True),True,False)
frame['HOME_PLAYER_IDS']=np.where(frame['QUARTER_START']==True,frame['HOME_PLAYER_IDS'],np.nan)
frame['AWAY_PLAYER_IDS']=np.where(frame['QUARTER_START']==True,frame['AWAY_PLAYER_IDS'],np.nan)
frame

,GAME_ID,EVENTNUM,EVENTMSGTYPE,EVENTMSGACTIONTYPE,PERIOD,WCTIMESTRING,PCTIMESTRING,HOMEDESCRIPTION,NEUTRALDESCRIPTION,VISITORDESCRIPTION,SCORE,SCOREMARGIN,PERSON1TYPE,PLAYER1_ID,PLAYER1_NAME,PLAYER1_TEAM_ID,PLAYER1_TEAM_CITY,PLAYER1_TEAM_NICKNAME,PLAYER1_TEAM_ABBREVIATION,PERSON2TYPE,PLAYER2_ID,PLAYER2_NAME,PLAYER2_TEAM_ID,PLAYER2_TEAM_CITY,PLAYER2_TEAM_NICKNAME,PLAYER2_TEAM_ABBREVIATION,PERSON3TYPE,PLAYER3_ID,PLAYER3_NAME,PLAYER3_TEAM_ID,PLAYER3_TEAM_CITY,PLAYER3_TEAM_NICKNAME,PLAYER3_TEAM_ABBREVIATION,VIDEO_AVAILABLE_FLAG,HOME,AWAY,HOME_PLAYER_IDS,AWAY_PLAYER_IDS,HOME_SUB,AWAY_SUB,QUARTER_START,PLAYER_CHANGE
0,0042100236,2,12,0,1,10:22 PM,12:00,,Start of 1st Period (10:22 PM EST),,,,0,0,,,,,,0,0,,,,,,0,0,,,,,,0,GSW,MEM,"203952,203110,1626172,202691,201939","1628415,1628991,203500,1630217,1626145",False,False,True,True
1,0042100236,4,10,0,1,10:22 PM,12:00,Jump Ball Looney vs. Adams: Tip to Jones,,,,,4,1626172,Kevon Looney,1610612744.0,Golden State,Warriors,GSW,5,203500,Steven Adams,1610612763.0,Memphis,Grizzlies,MEM,5,1626145,Tyus Jones,1610612763.0,Memphis,Grizzlies,MEM,1,GSW,MEM,NaN,NaN,False,False,False,False
2,0042100236,7,2,47,1,10:22 PM,11:40,,,MISS Jackson Jr. 9' Turnaround Jump Shot,,,5,1628991,Jaren Jackson Jr.,1610612763.0,Memphis,Grizzlies,MEM,0,0,,,,,,0,0,,,,,,1,GSW,MEM,NaN,NaN,False,False,False,False
3,0042100236,8,4,0,1,10:22 PM,11:37,,,Grizzlies Rebound,,,3,1610612763,,,,,,0,0,,,,,,0,0,,,,,,0,GSW,MEM,NaN,NaN,False,False,False,False
4,0042100236,9,6,3,1,10:22 PM,11:37,Green L.B.FOUL (P1.T1) (J.Williams),,,,,4,203110,Draymond Green,1610612744.0,Golden State,Warriors,GSW,5,203500,Steven Adams,1610612763.0,Memphis,Grizzlies,MEM,1,0,,,,,,1,GSW,MEM,NaN,NaN,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
479,0042100236,670,4,0,4,12:38 AM,0:30,,,Clarke REBOUND (Off:4 Def:0),,,5,1629634,Brandon Clarke,1610612763.0,Memphis,Grizzlies,MEM,0,0,,,,,,0,0,,,,,,1,GSW,MEM,NaN,NaN,False,False,False,False
480,0042100236,671,2,1,4,12:38 AM,0:27,,,MISS Jackson Jr. 29' 3PT Jump Shot,,,5,1628991,Jaren Jackson Jr.,1610612763.0,Memphis,Grizzlies,MEM,0,0,,,,,,0,0,,,,,,1,GSW,MEM,NaN,NaN,False,False,False,False
481,0042100236,672,4,0,4,12:38 AM,0:25,Curry REBOUND (Off:0 Def:7),,,,,4,201939,Stephen Curry,1610612744.0,Golden State,Warriors,GSW,0,0,,,,,,0,0,,,,,,1,GSW,MEM,NaN,NaN,False,False,False,False
482,0042100236,673,5,11,4,12:38 AM,0:01,WARRIORS Turnover: Shot Clock (T#19),,,,,2,1610612744,,,,,,0,0,,,,,,1,0,,,,,,0,GSW,MEM,NaN,NaN,False,False,False,False


In [9]:
for idx, row in frame.iterrows():
    if idx == 0 or row['QUARTER_START']==True:
        continue
    else:
        frame.at[idx,'AWAY_PLAYER_IDS']=frame.at[idx-1,'AWAY_PLAYER_IDS']
        frame.at[idx,'HOME_PLAYER_IDS']=frame.at[idx-1,'HOME_PLAYER_IDS']
        if row['AWAY_SUB']==True:
            frame.at[idx,'AWAY_PLAYER_IDS']=frame.at[idx,'AWAY_PLAYER_IDS'].replace(str(row['PLAYER1_ID']),str(row['PLAYER2_ID']))
        if row['HOME_SUB']==True:
            frame.at[idx,'HOME_PLAYER_IDS']=frame.at[idx,'HOME_PLAYER_IDS'].replace(str(row['PLAYER1_ID']),str(row['PLAYER2_ID']))
frame['HOME_PLAYER_IDS'].ffill(inplace=True)
frame['AWAY_PLAYER_IDS'].ffill(inplace=True)
frame

,GAME_ID,EVENTNUM,EVENTMSGTYPE,EVENTMSGACTIONTYPE,PERIOD,WCTIMESTRING,PCTIMESTRING,HOMEDESCRIPTION,NEUTRALDESCRIPTION,VISITORDESCRIPTION,SCORE,SCOREMARGIN,PERSON1TYPE,PLAYER1_ID,PLAYER1_NAME,PLAYER1_TEAM_ID,PLAYER1_TEAM_CITY,PLAYER1_TEAM_NICKNAME,PLAYER1_TEAM_ABBREVIATION,PERSON2TYPE,PLAYER2_ID,PLAYER2_NAME,PLAYER2_TEAM_ID,PLAYER2_TEAM_CITY,PLAYER2_TEAM_NICKNAME,PLAYER2_TEAM_ABBREVIATION,PERSON3TYPE,PLAYER3_ID,PLAYER3_NAME,PLAYER3_TEAM_ID,PLAYER3_TEAM_CITY,PLAYER3_TEAM_NICKNAME,PLAYER3_TEAM_ABBREVIATION,VIDEO_AVAILABLE_FLAG,HOME,AWAY,HOME_PLAYER_IDS,AWAY_PLAYER_IDS,HOME_SUB,AWAY_SUB,QUARTER_START,PLAYER_CHANGE
0,0042100236,2,12,0,1,10:22 PM,12:00,,Start of 1st Period (10:22 PM EST),,,,0,0,,,,,,0,0,,,,,,0,0,,,,,,0,GSW,MEM,"203952,203110,1626172,202691,201939","1628415,1628991,203500,1630217,1626145",False,False,True,True
1,0042100236,4,10,0,1,10:22 PM,12:00,Jump Ball Looney vs. Adams: Tip to Jones,,,,,4,1626172,Kevon Looney,1610612744.0,Golden State,Warriors,GSW,5,203500,Steven Adams,1610612763.0,Memphis,Grizzlies,MEM,5,1626145,Tyus Jones,1610612763.0,Memphis,Grizzlies,MEM,1,GSW,MEM,"203952,203110,1626172,202691,201939","1628415,1628991,203500,1630217,1626145",False,False,False,False
2,0042100236,7,2,47,1,10:22 PM,11:40,,,MISS Jackson Jr. 9' Turnaround Jump Shot,,,5,1628991,Jaren Jackson Jr.,1610612763.0,Memphis,Grizzlies,MEM,0,0,,,,,,0,0,,,,,,1,GSW,MEM,"203952,203110,1626172,202691,201939","1628415,1628991,203500,1630217,1626145",False,False,False,False
3,0042100236,8,4,0,1,10:22 PM,11:37,,,Grizzlies Rebound,,,3,1610612763,,,,,,0,0,,,,,,0,0,,,,,,0,GSW,MEM,"203952,203110,1626172,202691,201939","1628415,1628991,203500,1630217,1626145",False,False,False,False
4,0042100236,9,6,3,1,10:22 PM,11:37,Green L.B.FOUL (P1.T1) (J.Williams),,,,,4,203110,Draymond Green,1610612744.0,Golden State,Warriors,GSW,5,203500,Steven Adams,1610612763.0,Memphis,Grizzlies,MEM,1,0,,,,,,1,GSW,MEM,"203952,203110,1626172,202691,201939","1628415,1628991,203500,1630217,1626145",False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
479,0042100236,670,4,0,4,12:38 AM,0:30,,,Clarke REBOUND (Off:4 Def:0),,,5,1629634,Brandon Clarke,1610612763.0,Memphis,Grizzlies,MEM,0,0,,,,,,0,0,,,,,,1,GSW,MEM,"201939,1626172,202691,203952,203110","1629634,1630217,1628991,1626145,1628415",False,False,False,False
480,0042100236,671,2,1,4,12:38 AM,0:27,,,MISS Jackson Jr. 29' 3PT Jump Shot,,,5,1628991,Jaren Jackson Jr.,1610612763.0,Memphis,Grizzlies,MEM,0,0,,,,,,0,0,,,,,,1,GSW,MEM,"201939,1626172,202691,203952,203110","1629634,1630217,1628991,1626145,1628415",False,False,False,False
481,0042100236,672,4,0,4,12:38 AM,0:25,Curry REBOUND (Off:0 Def:7),,,,,4,201939,Stephen Curry,1610612744.0,Golden State,Warriors,GSW,0,0,,,,,,0,0,,,,,,1,GSW,MEM,"201939,1626172,202691,203952,203110","1629634,1630217,1628991,1626145,1628415",False,False,False,False
482,0042100236,673,5,11,4,12:38 AM,0:01,WARRIORS Turnover: Shot Clock (T#19),,,,,2,1610612744,,,,,,0,0,,,,,,1,0,,,,,,0,GSW,MEM,"201939,1626172,202691,203952,203110","1629634,1630217,1628991,1626145,1628415",False,False,False,False


In [10]:
frame=frame[frame['PLAYER_CHANGE']==False]
frame=frame[frame['NEUTRALDESCRIPTION']=='']
frame=frame[(~frame['HOMEDESCRIPTION'].str.contains('Jump Ball'))&(~frame['VISITORDESCRIPTION'].str.contains('Jump Ball'))]
frame=frame[(~frame['HOMEDESCRIPTION'].str.contains('Timeout'))&(~frame['VISITORDESCRIPTION'].str.contains('Timeout'))]
frame

,GAME_ID,EVENTNUM,EVENTMSGTYPE,EVENTMSGACTIONTYPE,PERIOD,WCTIMESTRING,PCTIMESTRING,HOMEDESCRIPTION,NEUTRALDESCRIPTION,VISITORDESCRIPTION,SCORE,SCOREMARGIN,PERSON1TYPE,PLAYER1_ID,PLAYER1_NAME,PLAYER1_TEAM_ID,PLAYER1_TEAM_CITY,PLAYER1_TEAM_NICKNAME,PLAYER1_TEAM_ABBREVIATION,PERSON2TYPE,PLAYER2_ID,PLAYER2_NAME,PLAYER2_TEAM_ID,PLAYER2_TEAM_CITY,PLAYER2_TEAM_NICKNAME,PLAYER2_TEAM_ABBREVIATION,PERSON3TYPE,PLAYER3_ID,PLAYER3_NAME,PLAYER3_TEAM_ID,PLAYER3_TEAM_CITY,PLAYER3_TEAM_NICKNAME,PLAYER3_TEAM_ABBREVIATION,VIDEO_AVAILABLE_FLAG,HOME,AWAY,HOME_PLAYER_IDS,AWAY_PLAYER_IDS,HOME_SUB,AWAY_SUB,QUARTER_START,PLAYER_CHANGE
2,0042100236,7,2,47,1,10:22 PM,11:40,,,MISS Jackson Jr. 9' Turnaround Jump Shot,,,5,1628991,Jaren Jackson Jr.,1610612763.0,Memphis,Grizzlies,MEM,0,0,,,,,,0,0,,,,,,1,GSW,MEM,"203952,203110,1626172,202691,201939","1628415,1628991,203500,1630217,1626145",False,False,False,False
3,0042100236,8,4,0,1,10:22 PM,11:37,,,Grizzlies Rebound,,,3,1610612763,,,,,,0,0,,,,,,0,0,,,,,,0,GSW,MEM,"203952,203110,1626172,202691,201939","1628415,1628991,203500,1630217,1626145",False,False,False,False
4,0042100236,9,6,3,1,10:22 PM,11:37,Green L.B.FOUL (P1.T1) (J.Williams),,,,,4,203110,Draymond Green,1610612744.0,Golden State,Warriors,GSW,5,203500,Steven Adams,1610612763.0,Memphis,Grizzlies,MEM,1,0,,,,,,1,GSW,MEM,"203952,203110,1626172,202691,201939","1628415,1628991,203500,1630217,1626145",False,False,False,False
5,0042100236,11,2,1,1,10:23 PM,11:31,,,MISS Bane 27' 3PT Jump Shot,,,5,1630217,Desmond Bane,1610612763.0,Memphis,Grizzlies,MEM,0,0,,,,,,0,0,,,,,,1,GSW,MEM,"203952,203110,1626172,202691,201939","1628415,1628991,203500,1630217,1626145",False,False,False,False
6,0042100236,12,4,0,1,10:23 PM,11:29,Green REBOUND (Off:0 Def:1),,,,,4,203110,Draymond Green,1610612744.0,Golden State,Warriors,GSW,0,0,,,,,,0,0,,,,,,1,GSW,MEM,"203952,203110,1626172,202691,201939","1628415,1628991,203500,1630217,1626145",False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
478,0042100236,669,2,1,4,12:38 AM,0:33,,,MISS Clarke 9' Jump Shot,,,5,1629634,Brandon Clarke,1610612763.0,Memphis,Grizzlies,MEM,0,0,,,,,,0,0,,,,,,1,GSW,MEM,"201939,1626172,202691,203952,203110","1629634,1630217,1628991,1626145,1628415",False,False,False,False
479,0042100236,670,4,0,4,12:38 AM,0:30,,,Clarke REBOUND (Off:4 Def:0),,,5,1629634,Brandon Clarke,1610612763.0,Memphis,Grizzlies,MEM,0,0,,,,,,0,0,,,,,,1,GSW,MEM,"201939,1626172,202691,203952,203110","1629634,1630217,1628991,1626145,1628415",False,False,False,False
480,0042100236,671,2,1,4,12:38 AM,0:27,,,MISS Jackson Jr. 29' 3PT Jump Shot,,,5,1628991,Jaren Jackson Jr.,1610612763.0,Memphis,Grizzlies,MEM,0,0,,,,,,0,0,,,,,,1,GSW,MEM,"201939,1626172,202691,203952,203110","1629634,1630217,1628991,1626145,1628415",False,False,False,False
481,0042100236,672,4,0,4,12:38 AM,0:25,Curry REBOUND (Off:0 Def:7),,,,,4,201939,Stephen Curry,1610612744.0,Golden State,Warriors,GSW,0,0,,,,,,0,0,,,,,,1,GSW,MEM,"201939,1626172,202691,203952,203110","1629634,1630217,1628991,1626145,1628415",False,False,False,False


In [11]:
frame['NEXT_HOME_PLAY']=frame['HOMEDESCRIPTION'].shift(-1)
frame['NEXT_AWAY_PLAY']=frame['VISITORDESCRIPTION'].shift(-1)
frame['TIMER']=frame['PCTIMESTRING'].str.split(':').str.get(0).astype(int)+frame['PCTIMESTRING'].str.split(':').str.get(1).astype(int)/60
frame['SECONDS_PASSED']=np.where(frame['TIMER'].shift(1)>=frame['TIMER'],frame['TIMER'].shift(1)*60-frame['TIMER']*60,(np.ceil(frame['TIMER'])-frame['TIMER'])*60)
frame['HOME_POSSESSION_FLAG']=np.where((frame['NEXT_HOME_PLAY']=='')&(frame['HOMEDESCRIPTION']!=''),1,0)
frame['AWAY_POSSESSION_FLAG']=np.where((frame['NEXT_AWAY_PLAY']=='')&(frame['VISITORDESCRIPTION']!=''),1,0)
frame

,GAME_ID,EVENTNUM,EVENTMSGTYPE,EVENTMSGACTIONTYPE,PERIOD,WCTIMESTRING,PCTIMESTRING,HOMEDESCRIPTION,NEUTRALDESCRIPTION,VISITORDESCRIPTION,SCORE,SCOREMARGIN,PERSON1TYPE,PLAYER1_ID,PLAYER1_NAME,PLAYER1_TEAM_ID,PLAYER1_TEAM_CITY,PLAYER1_TEAM_NICKNAME,PLAYER1_TEAM_ABBREVIATION,PERSON2TYPE,PLAYER2_ID,PLAYER2_NAME,PLAYER2_TEAM_ID,PLAYER2_TEAM_CITY,PLAYER2_TEAM_NICKNAME,PLAYER2_TEAM_ABBREVIATION,PERSON3TYPE,PLAYER3_ID,PLAYER3_NAME,PLAYER3_TEAM_ID,PLAYER3_TEAM_CITY,PLAYER3_TEAM_NICKNAME,PLAYER3_TEAM_ABBREVIATION,VIDEO_AVAILABLE_FLAG,HOME,AWAY,HOME_PLAYER_IDS,AWAY_PLAYER_IDS,HOME_SUB,AWAY_SUB,QUARTER_START,PLAYER_CHANGE,NEXT_HOME_PLAY,NEXT_AWAY_PLAY,TIMER,SECONDS_PASSED,HOME_POSSESSION_FLAG,AWAY_POSSESSION_FLAG
2,0042100236,7,2,47,1,10:22 PM,11:40,,,MISS Jackson Jr. 9' Turnaround Jump Shot,,,5,1628991,Jaren Jackson Jr.,1610612763.0,Memphis,Grizzlies,MEM,0,0,,,,,,0,0,,,,,,1,GSW,MEM,"203952,203110,1626172,202691,201939","1628415,1628991,203500,1630217,1626145",False,False,False,False,,Grizzlies Rebound,11.666667,20.0,0,0
3,0042100236,8,4,0,1,10:22 PM,11:37,,,Grizzlies Rebound,,,3,1610612763,,,,,,0,0,,,,,,0,0,,,,,,0,GSW,MEM,"203952,203110,1626172,202691,201939","1628415,1628991,203500,1630217,1626145",False,False,False,False,Green L.B.FOUL (P1.T1) (J.Williams),,11.616667,3.0,0,1
4,0042100236,9,6,3,1,10:22 PM,11:37,Green L.B.FOUL (P1.T1) (J.Williams),,,,,4,203110,Draymond Green,1610612744.0,Golden State,Warriors,GSW,5,203500,Steven Adams,1610612763.0,Memphis,Grizzlies,MEM,1,0,,,,,,1,GSW,MEM,"203952,203110,1626172,202691,201939","1628415,1628991,203500,1630217,1626145",False,False,False,False,,MISS Bane 27' 3PT Jump Shot,11.616667,0.0,1,0
5,0042100236,11,2,1,1,10:23 PM,11:31,,,MISS Bane 27' 3PT Jump Shot,,,5,1630217,Desmond Bane,1610612763.0,Memphis,Grizzlies,MEM,0,0,,,,,,0,0,,,,,,1,GSW,MEM,"203952,203110,1626172,202691,201939","1628415,1628991,203500,1630217,1626145",False,False,False,False,Green REBOUND (Off:0 Def:1),,11.516667,6.0,0,1
6,0042100236,12,4,0,1,10:23 PM,11:29,Green REBOUND (Off:0 Def:1),,,,,4,203110,Draymond Green,1610612744.0,Golden State,Warriors,GSW,0,0,,,,,,0,0,,,,,,1,GSW,MEM,"203952,203110,1626172,202691,201939","1628415,1628991,203500,1630217,1626145",False,False,False,False,MISS Wiggins 26' 3PT Jump Shot,,11.483333,2.0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
478,0042100236,669,2,1,4,12:38 AM,0:33,,,MISS Clarke 9' Jump Shot,,,5,1629634,Brandon Clarke,1610612763.0,Memphis,Grizzlies,MEM,0,0,,,,,,0,0,,,,,,1,GSW,MEM,"201939,1626172,202691,203952,203110","1629634,1630217,1628991,1626145,1628415",False,False,False,False,,Clarke REBOUND (Off:4 Def:0),0.550000,14.0,0,0
479,0042100236,670,4,0,4,12:38 AM,0:30,,,Clarke REBOUND (Off:4 Def:0),,,5,1629634,Brandon Clarke,1610612763.0,Memphis,Grizzlies,MEM,0,0,,,,,,0,0,,,,,,1,GSW,MEM,"201939,1626172,202691,203952,203110","1629634,1630217,1628991,1626145,1628415",False,False,False,False,,MISS Jackson Jr. 29' 3PT Jump Shot,0.500000,3.0,0,0
480,0042100236,671,2,1,4,12:38 AM,0:27,,,MISS Jackson Jr. 29' 3PT Jump Shot,,,5,1628991,Jaren Jackson Jr.,1610612763.0,Memphis,Grizzlies,MEM,0,0,,,,,,0,0,,,,,,1,GSW,MEM,"201939,1626172,202691,203952,203110","1629634,1630217,1628991,1626145,1628415",False,False,False,False,Curry REBOUND (Off:0 Def:7),,0.450000,3.0,0,1
481,0042100236,672,4,0,4,12:38 AM,0:25,Curry REBOUND (Off:0 Def:7),,,,,4,201939,Stephen Curry,1610612744.0,Golden State,Warriors,GSW,0,0,,,,,,0,0,,,,,,1,GSW,MEM,"201939,1626172,202691,203952,203110","1629634,1630217,1628991,1626145,1628415",False,False,False,False,WARRIORS Turnover: Shot Clock (T#19),,0.416667,2.0,0,0


In [13]:
frame['AWAY_SCORE']=frame['SCORE'].str.split(' - ').str.get(0)
frame['AWAY_SCORE']=np.where(frame['AWAY_SCORE']=='',np.nan,frame['AWAY_SCORE'])
frame['AWAY_SCORE'].ffill(inplace=True)
frame['AWAY_SCORE'].fillna(0,inplace=True)
frame['HOME_SCORE']=frame['SCORE'].str.split(' - ').str.get(1)
frame['HOME_SCORE'].ffill(inplace=True)
frame['HOME_SCORE'].fillna(0,inplace=True)
frame['HOME_SCORE_DIFFERENCE']=frame['HOME_SCORE'].astype(int)-frame['HOME_SCORE'].astype(int).shift(1)
frame['AWAY_SCORE_DIFFERENCE']=frame['AWAY_SCORE'].astype(int)-frame['AWAY_SCORE'].astype(int).shift(1)
frame['HOME_SCORE_DIFFERENCE'].fillna(0,inplace=True)
frame['AWAY_SCORE_DIFFERENCE'].fillna(0,inplace=True)
frame['HOME_VS_AWAY']=frame['HOME_SCORE_DIFFERENCE']-frame['AWAY_SCORE_DIFFERENCE']
frame

,GAME_ID,EVENTNUM,EVENTMSGTYPE,EVENTMSGACTIONTYPE,PERIOD,WCTIMESTRING,PCTIMESTRING,HOMEDESCRIPTION,NEUTRALDESCRIPTION,VISITORDESCRIPTION,SCORE,SCOREMARGIN,PERSON1TYPE,PLAYER1_ID,PLAYER1_NAME,PLAYER1_TEAM_ID,PLAYER1_TEAM_CITY,PLAYER1_TEAM_NICKNAME,PLAYER1_TEAM_ABBREVIATION,PERSON2TYPE,PLAYER2_ID,PLAYER2_NAME,PLAYER2_TEAM_ID,PLAYER2_TEAM_CITY,PLAYER2_TEAM_NICKNAME,PLAYER2_TEAM_ABBREVIATION,PERSON3TYPE,PLAYER3_ID,PLAYER3_NAME,PLAYER3_TEAM_ID,PLAYER3_TEAM_CITY,PLAYER3_TEAM_NICKNAME,PLAYER3_TEAM_ABBREVIATION,VIDEO_AVAILABLE_FLAG,HOME,AWAY,HOME_PLAYER_IDS,AWAY_PLAYER_IDS,HOME_SUB,AWAY_SUB,QUARTER_START,PLAYER_CHANGE,NEXT_HOME_PLAY,NEXT_AWAY_PLAY,TIMER,SECONDS_PASSED,HOME_POSSESSION_FLAG,AWAY_POSSESSION_FLAG,AWAY_SCORE,HOME_SCORE,HOME_SCORE_DIFFERENCE,AWAY_SCORE_DIFFERENCE,HOME_VS_AWAY
2,0042100236,7,2,47,1,10:22 PM,11:40,,,MISS Jackson Jr. 9' Turnaround Jump Shot,,,5,1628991,Jaren Jackson Jr.,1610612763.0,Memphis,Grizzlies,MEM,0,0,,,,,,0,0,,,,,,1,GSW,MEM,"203952,203110,1626172,202691,201939","1628415,1628991,203500,1630217,1626145",False,False,False,False,,Grizzlies Rebound,11.666667,20.0,0,0,0,0,0.0,0.0,0.0
3,0042100236,8,4,0,1,10:22 PM,11:37,,,Grizzlies Rebound,,,3,1610612763,,,,,,0,0,,,,,,0,0,,,,,,0,GSW,MEM,"203952,203110,1626172,202691,201939","1628415,1628991,203500,1630217,1626145",False,False,False,False,Green L.B.FOUL (P1.T1) (J.Williams),,11.616667,3.0,0,1,0,0,0.0,0.0,0.0
4,0042100236,9,6,3,1,10:22 PM,11:37,Green L.B.FOUL (P1.T1) (J.Williams),,,,,4,203110,Draymond Green,1610612744.0,Golden State,Warriors,GSW,5,203500,Steven Adams,1610612763.0,Memphis,Grizzlies,MEM,1,0,,,,,,1,GSW,MEM,"203952,203110,1626172,202691,201939","1628415,1628991,203500,1630217,1626145",False,False,False,False,,MISS Bane 27' 3PT Jump Shot,11.616667,0.0,1,0,0,0,0.0,0.0,0.0
5,0042100236,11,2,1,1,10:23 PM,11:31,,,MISS Bane 27' 3PT Jump Shot,,,5,1630217,Desmond Bane,1610612763.0,Memphis,Grizzlies,MEM,0,0,,,,,,0,0,,,,,,1,GSW,MEM,"203952,203110,1626172,202691,201939","1628415,1628991,203500,1630217,1626145",False,False,False,False,Green REBOUND (Off:0 Def:1),,11.516667,6.0,0,1,0,0,0.0,0.0,0.0
6,0042100236,12,4,0,1,10:23 PM,11:29,Green REBOUND (Off:0 Def:1),,,,,4,203110,Draymond Green,1610612744.0,Golden State,Warriors,GSW,0,0,,,,,,0,0,,,,,,1,GSW,MEM,"203952,203110,1626172,202691,201939","1628415,1628991,203500,1630217,1626145",False,False,False,False,MISS Wiggins 26' 3PT Jump Shot,,11.483333,2.0,0,0,0,0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
478,0042100236,669,2,1,4,12:38 AM,0:33,,,MISS Clarke 9' Jump Shot,,,5,1629634,Brandon Clarke,1610612763.0,Memphis,Grizzlies,MEM,0,0,,,,,,0,0,,,,,,1,GSW,MEM,"201939,1626172,202691,203952,203110","1629634,1630217,1628991,1626145,1628415",False,False,False,False,,Clarke REBOUND (Off:4 Def:0),0.550000,14.0,0,0,96,110,0.0,0.0,0.0
479,0042100236,670,4,0,4,12:38 AM,0:30,,,Clarke REBOUND (Off:4 Def:0),,,5,1629634,Brandon Clarke,1610612763.0,Memphis,Grizzlies,MEM,0,0,,,,,,0,0,,,,,,1,GSW,MEM,"201939,1626172,202691,203952,203110","1629634,1630217,1628991,1626145,1628415",False,False,False,False,,MISS Jackson Jr. 29' 3PT Jump Shot,0.500000,3.0,0,0,96,110,0.0,0.0,0.0
480,0042100236,671,2,1,4,12:38 AM,0:27,,,MISS Jackson Jr. 29' 3PT Jump Shot,,,5,1628991,Jaren Jackson Jr.,1610612763.0,Memphis,Grizzlies,MEM,0,0,,,,,,0,0,,,,,,1,GSW,MEM,"201939,1626172,202691,203952,203110","1629634,1630217,1628991,1626145,1628415",False,False,False,False,Curry REBOUND (Off:0 Def:7),,0.450000,3.0,0,1,96,110,0.0,0.0,0.0
481,0042100236,672,4,0,4,12:38 AM,0:25,Curry REBOUND (Off:0 Def:7),,,,,4,201939,Stephen Curry,1610612744.0,Golden State,Warriors,GSW,0,0,,,,,,0,0,,,,,,1,GSW,MEM,"201939,1626172,202691,203952,203110","1629634,1630217,1628991,1626145,1628415",False,False,False,False,WARRIORS Turnover: Shot Clock (T#19),,0.416667,2.0,0,0,96,110,0.0,0.0,0.0
